In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
# read data
#df_raw = pd.read_csv(r"C:\Users\GTR\Documents\GitHub\cars_sales\raw_data\vehicles.csv")

## 1. Extract

In [2]:
df = pd.read_csv(r"C:\Users\GTR\Documents\GitHub\cars_sales\ETL_data\vehicles_clean.csv")

## 2. Transforme

### 2.1 Transforming and cleaning data

In [5]:
# romove unreal price 
df = df.loc[df['price'] >= 1000]
# # drop useless columns
df.drop(['id','url','region_url','image_url','VIN','size','county','lat','long','drive'], axis=1,inplace=True)
#drop nan rows
df.dropna(subset=(['year','manufacturer','model','paint_color','condition','cylinders','odometer','transmission','type','price']), inplace=True)
# add kilometre column
df['kilometre'] = df['odometer'] * 1.60934
# add cylinders column with int type
df['cylinders'] = df['cylinders'].str.replace('cylinders','').str.strip()

In [6]:
#Checking Null Values
df.isnull().sum()

region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
type            0
paint_color     0
description     0
state           0
posting_date    0
kilometre       0
dtype: int64

### 2.2 Preparing tables

### Time Dimension

In [3]:
D_time = df[['year']].drop_duplicates().sort_values('year').reset_index(drop=True)
D_time['id_time'] = D_time.index
D_time = D_time[['id_time','year']]
D_time.head(2)

,id_time,year
0,0,1900.0
1,1,1905.0


### Location Dimension

In [4]:
D_location = df[['state','region']].drop_duplicates().sort_values('state').reset_index(drop=True)
D_location['id_location'] = D_location.index
D_location = D_location[['id_location','state','region']]
D_location.head(2)

,id_location,state,region
0,0,ak,anchorage / mat-su
1,1,ak,southeast alaska


### Title status Dimension

In [5]:
D_title_status = df[['title_status']].drop_duplicates().sort_values('title_status').reset_index(drop=True)
D_title_status['id_title_status'] = D_title_status.index
D_title_status = D_title_status[['id_title_status','title_status']]
D_title_status.head(2)

,id_title_status,title_status
0,0,clean
1,1,lien


### Paint color Dimension

In [6]:
D_paint_color = df[['paint_color']].drop_duplicates().sort_values('paint_color').reset_index(drop=True)
D_paint_color['id_paint_color'] = D_paint_color.index
D_paint_color = D_paint_color[['id_paint_color','paint_color']]
D_paint_color.head(2)

,id_paint_color,paint_color
0,0,black
1,1,blue


### car Dimension

In [7]:
D_car = df[['manufacturer', 'type']].drop_duplicates().sort_values(['manufacturer', 'type']).reset_index(drop=True)
D_car['id_car'] = D_car.index
D_car = D_car[['id_car','manufacturer', 'type']]
D_car.head(2)

,id_car,manufacturer,type
0,0,acura,SUV
1,1,acura,coupe


### Condition Dimension 

In [8]:
D_condition = df[['condition']].drop_duplicates().sort_values('condition').reset_index(drop=True)
D_condition['id_condition'] = D_condition.index
D_condition = D_condition[['id_condition','condition']]
D_condition.head(2)

,id_condition,condition
0,0,excellent
1,1,fair


### Engine Dimension

In [9]:
D_engine = df[['cylinders', 'fuel', 'transmission']].drop_duplicates().sort_values(['cylinders', 'fuel', 'transmission']).reset_index(drop=True)
D_engine['id_engine'] = D_engine.index
D_engine = D_engine[['id_engine','cylinders', 'fuel', 'transmission']]
D_engine.head(2)

,id_engine,cylinders,fuel,transmission
0,0,10,diesel,automatic
1,1,10,gas,automatic


### Fact table

In [10]:
# merge fact table
fact_table = df.merge(D_time, on='year', how='left')\
.merge(D_location, on=['state','region'], how='left')\
.merge(D_title_status, on='title_status', how='left')\
.merge(D_paint_color, on='paint_color', how='left')\
.merge(D_car, on=['manufacturer', 'type'], how='left')\
.merge(D_condition, on='condition', how='left')\
.merge(D_engine, on=['cylinders', 'fuel', 'transmission'], how='left')

# add count column
fact_table['count'] = 1
# add mean price and mean kilometre column
fact_table['mean_price'] = fact_table['price']
fact_table['mean_kilometre'] = fact_table['kilometre']

# remove useless columns
fact_table = fact_table[['id_time','id_location','id_title_status','id_paint_color','id_car','id_condition','id_engine', 'price', 'mean_price', 'kilometre', 'mean_kilometre', 'count']]

# group by fact table
fact_table = fact_table.groupby(['id_time','id_location','id_title_status','id_paint_color','id_car','id_condition','id_engine']).agg({'price':'sum','kilometre':'sum','mean_price':'mean','mean_kilometre':'mean','count':'sum'}).reset_index()

## 3. Load

### 3.1 Load to csv_file

In [15]:
# df.to_csv(r"C:\Users\GTR\Documents\GitHub\cars_sales\ETL_data\vehicles_clean.csv", index=False)

### 3.2 Load to DB MS SQL

In [17]:
import pandas as pd
from sqlalchemy import create_engine 
from sqlalchemy.engine import URL
import pypyodbc as odbc

SERVER_NAME = 'Unknown1\SQLEXPRESS'
DATABASE_NAME = 'cars_sales'

connection_str = 'Driver={SQL Server};Server=Unknown1\SQLEXPRESS;Database=cars_sales;Trusted_Connection=yes;'
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_str})
engine = create_engine(connection_url, module=odbc)

In [ ]:
# Insert the DataFrame into the SQL Server table
D_time.to_sql('D_time', engine, index=False, if_exists='append') 
D_car.to_sql('D_car', engine, index=False, if_exists='append')
D_condition.to_sql('D_condition', engine, index=False, if_exists='append') 
D_engine.to_sql('D_engine', engine, index=False, if_exists='append')
D_location.to_sql('D_location', engine, index=False, if_exists='append') 
D_title_status.to_sql('D_title_status', engine, index=False, if_exists='append')
D_paint_color.to_sql('D_paint_color', engine, index=False, if_exists='append') 
fact_table.to_sql('Fact_table', engine, index=False, if_exists='append') 

103